# potsdam data pipeline

In [1]:
import os
import numpy as np
import pathlib
import re
from tqdm import tqdm 
from PIL import Image

In [3]:
# Specify all data file paths
d = './potsdam/'
dirs = {
    'in_imgs': d + 'tiff_imgs/',
    'in_masks': d + 'tiff_masks/',
    'out_imgs': d + 'imgs/',
    'out_masks': d + 'masks/',
    'out_split_list': '../dataset/potsdam_list'
}

In [4]:
# convert tiffs to png tiles. 38 images, 6000x6000   

# Splits tiff into tiles and saves each as png
def split_tiff(inpath, tile_pre, is_mask):
    # adapted from http://blog.odonnell.nu/posts/creating-tiles-image-python-and-pil/
    tile_ids = []
    indir = dirs['in_masks'] if is_mask else dirs['in_imgs']
    outdir = dirs['out_masks'] if is_mask else dirs['out_imgs']
    image = Image.open(indir + inpath)
    tile_width = 1000
    tile_height = 1000

    currentx = 0
    currenty = 0
    while currenty < image.size[1]:
        while currentx < image.size[0]:
#             tile = image.crop((currentx, currenty, currentx + tile_width, currenty + tile_height))
            ID = outdir + tile_pre.format(currentx, currenty)
            tile_ids.append(ID)
#             tile.save(ID + ".png", "PNG", quality=100)
            currentx += tile_width
        currenty += tile_height
        currentx = 0
    
    return tile_ids
    

# Tile and partition all imgs and masks
# 1368 tiles of 1000x1000. Train: 950, Val: 266, Test: 152
partition = {
    'train': [],
    'val': [],
    'test': [],
}

for tiff_type in ['imgs', 'masks']:
    if tiff_type == 'imgs':
        outdir, indir = dirs['out_imgs'], dirs['in_imgs']
        is_img = True # flag 
    else:
        outdir, indir = dirs['out_masks'], dirs['in_masks']
        is_img = False
    
    # tiff_paths = [p.as_posix() for p in pathlib.Path(indir).glob('*.tif')]
    dir_paths = os.listdir(indir)
    tiff_paths = [f for f in dir_paths if f.endswith(".tif")]
    
    for p in tqdm(tiff_paths):
        #if (p != 'top_potsdam_2_10_RGB.tif' and p != 'top_potsdam_2_10_label.tif'):
        #    continue
        
        m = re.search('_([0-9]*_[0-9]*)_', p) # extract image id (site #, pic #)
        tile_prefix = m.group(1) + '_{}_{}' # add formatters for tile row / col    
        tile_ids = split_tiff(p, tile_prefix, not is_img)
        
        # Partition tile ids into train/val/test folds
        if is_img:
            # shuffle ids
            rng = np.random.default_rng()
            rng.shuffle(tile_ids)
            # define partition splits
            num_tiles = len(tile_ids)
            pt1, pt2 = int(num_tiles*.7), int(num_tiles*.9)
            # assign to folds
            partition['train'] += tile_ids[:pt1]
            partition['val'] += tile_ids[pt1:pt2]
            partition['test'] += tile_ids[pt2:]

100%|██████████| 38/38 [00:00<00:00, 342.51it/s]


In [5]:
for split in partition.keys():
    with open('%s/%s.txt' % (dirs['out_split_list'], split), 'w') as file:
        for ID in partition[split]:
            file.write('%s.png\n' % ID.split('/')[-1])

# scratch

In [6]:
# old path scheme including folder
indir = dirs['in_imgs']
print(len([p.as_posix() for p in pathlib.Path(indir).glob('*.tif')]))


38


In [8]:
# new path scheme not including folder
dir_paths = os.listdir(indir)
tiff_paths = [f for f in dir_paths if f.endswith(".tif")]
print(len(tiff_paths))


38


In [34]:
# how to save as png in new folder
img_path = tiff_paths[0]
im = Image.open(img_path)
im.thumbnail(im.size)
outfile = dirs['out_imgs'] + 'blah' + '.png'
im.save(outfile, 'PNG', quality=100)


In [18]:
inpath = 'top_potsdam_2_10_RGB.tif'
indir = 'tiff_imgs/'
outdir = 'imgs/'
m = re.search('_([0-9]*_[0-9]*)_', inpath) # extract image id (site #, pic #)
tile_pre = m.group(1) + '_{}_{}' # add formatters for tile row / col 

tile_ids = []
with rio.open(d + indir + inpath) as tiff:
    tile_width, tile_height = 1000, 1000
    for window in get_tiles(tiff):
        ID = tile_pre.format(int(window.col_off), int(window.row_off))
        tile_ids.append(ID) # TODO
        
print(tile_ids)

['2_10_0_0', '2_10_0_1000', '2_10_0_2000', '2_10_0_3000', '2_10_0_4000', '2_10_0_5000', '2_10_1000_0', '2_10_1000_1000', '2_10_1000_2000', '2_10_1000_3000', '2_10_1000_4000', '2_10_1000_5000', '2_10_2000_0', '2_10_2000_1000', '2_10_2000_2000', '2_10_2000_3000', '2_10_2000_4000', '2_10_2000_5000', '2_10_3000_0', '2_10_3000_1000', '2_10_3000_2000', '2_10_3000_3000', '2_10_3000_4000', '2_10_3000_5000', '2_10_4000_0', '2_10_4000_1000', '2_10_4000_2000', '2_10_4000_3000', '2_10_4000_4000', '2_10_4000_5000', '2_10_5000_0', '2_10_5000_1000', '2_10_5000_2000', '2_10_5000_3000', '2_10_5000_4000', '2_10_5000_5000']


In [24]:
# adapted from http://blog.odonnell.nu/posts/creating-tiles-image-python-and-pil/

image = Image.open(dirs['in_imgs'] + inpath)
tile_width = 1000
tile_height = 1000

currentx = 0
currenty = 0
while currenty < image.size[1]:
    while currentx < image.size[0]:
        print(currentx, ",", currenty)
        tile = image.crop((currentx,currenty,currentx + tile_width,
                           currenty + tile_height))
        outfile = dirs['out_imgs'] + "blah" + str(currentx) + "_" + str(currenty) + ".png"
        tile.save(outfile, "PNG", quality=100)
        currentx += tile_width
    currenty += tile_height
    currentx = 0


0 , 0
1000 , 0
2000 , 0
3000 , 0
4000 , 0
5000 , 0
0 , 1000
1000 , 1000
2000 , 1000
3000 , 1000
4000 , 1000
5000 , 1000
0 , 2000
1000 , 2000
2000 , 2000
3000 , 2000
4000 , 2000
5000 , 2000
0 , 3000
1000 , 3000
2000 , 3000
3000 , 3000
4000 , 3000
5000 , 3000
0 , 4000
1000 , 4000
2000 , 4000
3000 , 4000
4000 , 4000
5000 , 4000
0 , 5000
1000 , 5000
2000 , 5000
3000 , 5000
4000 , 5000
5000 , 5000
